In [1]:
from hilde.watchdogs import WallTimeWatchdog
from time import sleep
from ase.md.verlet import VelocityVerlet
from ase.io import read
from hilde.templates.aims import setup_aims
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from ase.calculators.socketio import SocketIOCalculator

In [2]:
atoms = read('si.in', format='aims')

In [3]:
aims = setup_aims(custom_settings={'compute_forces': True}, port=12345)

In [4]:
T = 100 * units.kB

MaxwellBoltzmannDistribution(atoms, temp=T)

md = VelocityVerlet(atoms, timestep=4*units.fs, logfile='watchdog.log')

In [5]:
watchdog = WallTimeWatchdog(walltime=30, buffer=2)

with SocketIOCalculator(aims, port=12345) as calc:
    atoms.calc = calc
    for _ in md.irun(steps=1000):
        print(f'{watchdog.increment_per_step:.2f}, {watchdog.buffer_time:.2f}, ' +
              f'{watchdog.time_left:.2f} {watchdog.elapsed:.2f}')
        if watchdog():
            break

0.00, 0.00, 30.00 0.00
0.00, 0.00, 18.22 11.78
5.89, 11.78, 13.31 16.69
5.56, 11.12, 7.01 22.99
